In [1]:
from IPython.display import clear_output # 그림을 그리고 지우는 코드
!pip3 install -U lazypredict
!pip3 install -U # 판다스 업그레이드

clear_output()

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px # 파이썬의 시각화 라이브러리
import matplotlib.pyplot as plt
import plotly.graph_objects as go # 그래프 그리기
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer # 결측치 대치 파라미터
from sklearn.metrics import accuracy_score # 정확도 평가
from sklearn.preprocessing import LabelEncoder # 문자를 숫자로
from sklearn.model_selection import StratifiedKFold, train_test_split # 교차 검증

from lightgbm import LGBMClassifier
import lazypredict # 머신러닝 자동화 관련 파이썬 오픈소스
from lazypredict.Supervised import LazyClassifier

import time # 시간 데이터
import warnings # 경고 제어
warnings.filterwarnings('ignore') # 경고 메시지 무시하기

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5

  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12
FOLDS = 5
STRATEGY = 'median'

# imputation - strategy는 결측값을 채우기 위해 사용됨
# median 외에도 mean, most_frequent, constant가 기본적으로 제공됨

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/spaceship-titanic/train.csv'

In [ ]:
train.head()

In [ ]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}')
print(f'\033[94mNumber of columns in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber missing values in train data: {sum(train.isna().sum())}')

In [ ]:
 print(f'\033[94m')
print(train.isna().sum().sort_values(ascending = False)) # 내림차순

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values in test data: {sum(test.isna().sum())}')

In [ ]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False))) # 내림차순

In [ ]:
test.describe()

In [ ]:
submission.head()

In [ ]:
train.drop(["PassengerId"], axis = 1, inplace = True)
test.drop(["PassengerId"], axis = 1, inplace = True)
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col != TARGET]
RANDOM_STATE = 12

In [ ]:
#train.iloc[:, :-1].describe().T.sort_values(by='std', ascending = False)\
#.style.background_gradient(cmap='GuBu')\
#.bar(subset=["max"], color = '#BB0000')\
#.bar(subset=["mean"], color = 'grean')

In [ ]:
test_null = pd.DataFrame(test.isna().sum())
test_null = test_null.sort_values(by = 0, ascending = False)
train_null = pd.DataFrame(train.isna().sum())
train_null = train_null.sort_values(by=0, ascending = False)[:-1]

fig = make_subplots(rows=1, cols = 2, column_titles = ["Train Data", "Test Data"],
                   x_title = "Missing Values")

fig.add_trace(go.Bar(x=train_null[0], y=train_null.index, orientation="h",
                    marker=dict(color=[n for n in range(12)],
                                line_color = 'rgb(0,0,0)', line_width = 2,
                               coloraxis="coloraxis")), 1, 1)
fig.add_trace(go.Bar(x=test_null[0], y=test_null.index, orientation="h",
                    marker=dict(color=[n for n in range(12)],
                                line_color='rgb(0,0,0)', line_width = 2,
                               coloraxis="coloraxis")), 1, 2)

fig.update_layout(showlegend=False, title_text="Column wise Null Value Distribution", title_x = 0.5)

In [ ]:
missing_train_row = train.isna().sum(axis=1)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()
missing_test_row = test.isna().sum(axis=1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()
missing_train_row.columns = ['no', 'count']
missing_test_row.columns = ['no', 'count']
missing_train_row["count"] = missing_train_row["count"]*100
missing_test_row["count"] = missing_test_row["count"]*100

fig = make_subplots(rows=1, cols=2, column_titles = ["Train Data", "Test Data"],
                   x_title = "Missing Values")
fig.add_trace(go.Bar(x = missing_train_row["no"], y=missing_train_row["count"],
                    marker = dict(color = [n for n in range(4)], line_color = 'rgb(0,0,0)',
                                 line_width=3, coloraxis="coloraxis")), 1,1)
fig.add_trace(go.Bar(x = missing_test_row["no"], y=missing_test_row["count"],
                    marker = dict(color = [n for n in range(4)], line_color = 'rgb(0,0,0)',
                                 line_width=3, coloraxis = "coloraxis")), 1,2)
fig.update_layout(showlegend=False, title_text = "Row wise Null Value Distribution", title_x=0.5)

In [ ]:
df = pd.concat([train[FEATURES], test[FEATURES]], axis = 0)
text_features = ["Cabin", "Name"]
cat_features = [col for col in FEATURES if df[col].nunique() < 25 and col not in text_features]
cont_features = [col for col in FEATURES if df[col].nunique() >= 25 and col not in text_features]

del df
print(f'\033[94mTotal number of features: {len(FEATURES)}')
print(f'\033[94mNumber of categorical features: {len(cat_features)}')
print(f'\033[94mNumber of continuos features: {len(cont_features)}')
print(f'\033[94mNumber of text features: {len(text_features)}')

labels = ['Categorical', 'Continuos', "Text"]
values = [len(cat_features), len(cont_features), len(text_features)]
colors = ['#DE3163', '#58D68D']

fig = go.Figure(data = [go.Pie(labels=labels, values = values, pull = [0.1, 0, 0],
                              marker = dict(colors = colors, line=dict(color='#000000', width = 2)))])

fig.show()

In [ ]:
train_age = train.copy()
test_age = test.copy()
train_age["type"] = "Train"
test_age["type"] = "Test"
ageDf = pd.concat([train_age, test_age])
fig = px.histogram(data_frame = ageDf, x = "Age", color = "type", color_discrete_sequence = ['#58D68D', '#DE3163'],
                  marginal = "box", nbins=100, template="plotly_white")
fig.update_layout(title = "Distribution of Age", title_x = 0.5)
fig.show()

In [ ]:
if len(cat_features) == 0:
    print("No Categorical features")
else:
    ncols = 2
    nrows = 2
    
    fig, axes = plt.subplots(nrows, ncols, figsize = (18,10))
    for r in range(nrows):
        for c in range(ncols):
            col = cat_features[r*ncols+c]
            sns.countplot(train[col], ax=axes[r,c], x = train[col], palette="viridis")
            sns.countplot(test[col], ax=axes[r,c], x = test[col], palette="magma")
            axes[r,c].legend()
            axes[r,c].set_ylabel('')
            axes[r,c].set_xlabel(col, fontsize=20)
            axes[r,c].tick_params(labelsize=10, width=0.5)
            axes[r,c].xaxis.offsetText.set_fontsize(4)
            axes[r,c].yaxis.offsetText.set_fontsize(4)
        plt.show()

In [ ]:
target_df = pd.DataFrame(train[TARGET].value_counts()).reset_index()
target_df.columns = [TARGET, 'count']
fig = px.bar(data_frame = target_df, x=TARGET, y='count')
fig.update_traces(marker_color = ['#58D68D', '#DE3163'], marker_line_color = 'rgb(0,0,0)', marker_line_width=2, )

fig.update_layout(title = "Target Distribution", template = "plotly_white", title_x = 0.5)
print("\033[94mPercentage of Transported = 0: {:.2f} %".format(target_df["count"][0]*100/train.shape[0]))
print("\033[94mPercentage of Transported = 1: {:.2f} %".format(target_df["count"][1]*100/test.shape[0]))

In [ ]:
#fig = px.imshow(train.corr(), text_auto=True, aspect="auto", color_continuous_scale = "viridis")
#fig.show()

In [ ]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "RoomService"]
imputer = SimpleImputer(strategy = STRATEGY) # 결측값 중간값으로 채우는 설정
imputer.fit(train[imputer_cols]) # 중간값 계산
train[imputer_cols] = imputer.transform(train[imputer_cols]) # 변형된 특성 있는 배열 넣음
test[imputer_cols] = imputer.transform(test[imputer_cols])
train["HomePlanet"].fillna('Z', inplace = True) # NaN값 Z로 채우기
test["HomePlanet"].fillna('Z', inplace = True)

In [ ]:
label_cols = ["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP"]
def label_encoder(train, test, columns):
    for col in columns:
        train[col] = train[col].astype(str) # 문자형으로 변경
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] = LabelEncoder().fit_transform(test[col])
    return train, test

train, test = label_encoder(train, test, label_cols)

In [ ]:
train.drop(["Name", "Cabin"], axis = 1, inplace = True) # 버리기
test.drop(["Name", "Cabin"], axis = 1, inplace = True)
X = train.drop(TARGET, axis = 1)
y = train[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 12, test_size = 0.33)